# Testing
### Ehescheidungen nach Zahl minderjähriger Kinder, seit 1993
Datum: 02.11.2021

**Importiere die notwendigen Packages**

In [1]:
#pip install geopandas fiona requests folium mplleaflet contextily

In [5]:
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from datetime import datetime
import geopandas as gpd
import folium 



Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [6]:
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

Importiere einen Datensatz von der **DWH_INT** Dropzone.

- Beachte dabei die Notation des Pfades...
- Definiere mal aktuell noch keine weiteren Parameter beim Import

**Dataset auf INTEG Datenkatalog**: https://data.integ.stadt-zuerich.ch/dataset/int_dwh_bev_scheidungen_anz_miderjaehrige_kinder_od3304

(int_dwh)

In [13]:
#Die Datasets sind nur zum Testen auf INT-DWH-Dropzone. Wenn der Test vorbei ist, sind sie auf PROD. 
# Über den Status kann man einfach switchen

status = "prod";
print(status)

prod


In [15]:
# Filepath
if status == "prod":
    fp = r"\\szh\ssz\applikationen\OGD_Dropzone\DWH\bev_scheidungen_anz_miderjaehrige_kinder_od3304\BEV330OD3304.csv"
else:
    fp = r"\\szh\ssz\applikationen\OGD_Dropzone\INT_DWH\bev_scheidungen_anz_miderjaehrige_kinder_od3304\BEV330OD3304.csv"

In [14]:
# Read the data
data2betested = pd.read_csv(
    fp, 
    sep=',',
   #usecols=[0,1,2,3,4,6],
   #names=['stationid','year','month','avgtemp','latitude','longitude','elevation','station','countryid','country'],
   #skiprows=1,
   # parse_dates=[0],
    parse_dates=True,
    #date_parser=lambda x: pd.to_datetime(x, format="%Y"), 
    #index_col="date_time",
    low_memory=False    
    )


In der Folge ein paar erste Tests:

In [5]:
type(data2betested)

pandas.core.frame.DataFrame

 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [6]:
data2betested.head(3)

,StichtagDatJahr,AnzKinder,AnzKinderCd,AnzFaelle
0,1993,keine,0,967
1,1993,1,1,257
2,1993,2,2,217


In [7]:
data2betested.dtypes

StichtagDatJahr     int64
AnzKinder          object
AnzKinderCd         int64
AnzFaelle           int64
dtype: object

In [8]:
data2betested.shape

(224, 4)

Beschreibe einzelne Attribute

In [9]:
data2betested.describe()

,StichtagDatJahr,AnzKinderCd,AnzFaelle
count,224,224,224
mean,2006,4,186
std,8,2,349
min,1993,0,0
25%,2000,2,0
50%,2006,4,10
75%,2013,5,204
max,2020,7,1538


Wie viele Nullwerte gibt es im Datensatz?

In [10]:
data2betested.isnull().sum()

StichtagDatJahr    0
AnzKinder          0
AnzKinderCd        0
AnzFaelle          0
dtype: int64

Wandle die Spalte Jahr von Int zu **Datetime** um. Dies wird in den meisten Grafiken so benötigt.

In [11]:
data2betested['StichtagDatJahr']=pd.to_datetime(data2betested['StichtagDatJahr'], format='%Y')

#TO DO --> setze das Datum auf den 31.12. der Jahres.
#test=data2betested[['StichtagDatJahr']].date().replace(month=12, day=31)

data2betested

,StichtagDatJahr,AnzKinder,AnzKinderCd,AnzFaelle
0,1993-01-01,keine,0,967
1,1993-01-01,1,1,257
2,1993-01-01,2,2,217
3,1993-01-01,3,3,26
4,1993-01-01,4,4,8
...,...,...,...,...
219,2020-01-01,3,3,27
220,2020-01-01,4,4,2
221,2020-01-01,5,5,1
222,2020-01-01,6,6,0


Bereite die Daten für die Grafik unten vor

In [12]:
Jahre = data2betested.groupby("StichtagDatJahr")
len(Jahre)

28

In [13]:
for key, group in Jahre:
    print(key, len(group))

1993-01-01 00:00:00 8
1994-01-01 00:00:00 8
1995-01-01 00:00:00 8
1996-01-01 00:00:00 8
1997-01-01 00:00:00 8
1998-01-01 00:00:00 8
1999-01-01 00:00:00 8
2000-01-01 00:00:00 8
2001-01-01 00:00:00 8
2002-01-01 00:00:00 8
2003-01-01 00:00:00 8
2004-01-01 00:00:00 8
2005-01-01 00:00:00 8
2006-01-01 00:00:00 8
2007-01-01 00:00:00 8
2008-01-01 00:00:00 8
2009-01-01 00:00:00 8
2010-01-01 00:00:00 8
2011-01-01 00:00:00 8
2012-01-01 00:00:00 8
2013-01-01 00:00:00 8
2014-01-01 00:00:00 8
2015-01-01 00:00:00 8
2016-01-01 00:00:00 8
2017-01-01 00:00:00 8
2018-01-01 00:00:00 8
2019-01-01 00:00:00 8
2020-01-01 00:00:00 8


In [14]:
myJahr = "2020-01-01"
aktuellesJahr=Jahre.get_group(myJahr)   
aktuellesJahr

,StichtagDatJahr,AnzKinder,AnzKinderCd,AnzFaelle
216,2020-01-01,keine,0,772
217,2020-01-01,1,1,203
218,2020-01-01,2,2,153
219,2020-01-01,3,3,27
220,2020-01-01,4,4,2
221,2020-01-01,5,5,1
222,2020-01-01,6,6,0
223,2020-01-01,Unbekannt,7,0


**Selektionen und Aggregationen**

Check this [Doku](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/)

In [15]:
#test = data2betested.groupby(['StichtagDatJahr', 'AlterV20Kurz'])[['AnzBestWir']].agg()

# produces Pandas Series
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])['AnzBestWir'].sum()
# Produces Pandas DataFrame
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])[['AnzBestWir']].sum()
    

**Einfache Visualisierungen zur Plausi**

Exploriere die Daten mit Pivottable.JS

In [16]:
from pivottablejs import pivot_ui

pivot_ui(data2betested)

Liniendiagramm 
[Link zur Doku](https://altair-viz.github.io/gallery/multiline_highlight.html)

In [19]:
data2betested.head(2)

,StichtagDatJahr,AnzKinder,AnzKinderCd,AnzFaelle
0,1993-01-01,keine,0,967
1,1993-01-01,1,1,257


In [20]:
# df_statzonen = df_statzonen[(df_statzonen.knr>0) & (df_statzonen.qname!="Hirslanden")]
#mySelection[[mySelection.NationFHistLang =='Schweiz']]


highlight = alt.selection(type='single', on='mouseover',
                          fields=['AnzKinder'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(data2betested).encode(
    x='StichtagDatJahr',
    y='AnzFaelle',
    color='AnzKinder'
)

points = base.mark_circle().encode(
    opacity=alt.value(2)
).add_selection(
    highlight
).properties(
    width=700 , height=300
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
).interactive()

points + lines

alt.LayerChart(...)

In [21]:
alt.Chart(data2betested).mark_circle(size=60).encode(
    x='StichtagDatJahr',
    y='AnzFaelle',
    #color='AnzNat',
    #color=alt.Color('AnzNat', scale=alt.Scale(scheme='greens')),
    color=alt.Color('AnzKinder', scale=alt.Scale(scheme='dark2')),
    tooltip=['StichtagDatJahr','AnzKinder','AnzFaelle']
).interactive() # this makes the axes interactive: now you can zoom & pan




alt.Chart(...)

**Sharepoint als gecheckt markieren!**

Record auf Sharepoint: **[Link](http://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/EditForm.aspx?ID=52&Source=%2Forga%2Fssz%2Dprodukte%2FLists%2FSASA%5FOutputs)**